# Creating and consuming rss feeds.

    !pip install feedparser feedgen

In [11]:
    from feedgen.feed import FeedGenerator
    import feedparser
    from pathlib import Path
    import ipywidgets as W
    from IPython.display import display, Markdown
    from collections import UserDict
    import deathbeds
    import pandas
    from sidecar import Sidecar
    from deathbeds.__summarizing_notebooks_with_gensim import summarize

In [3]:
feed = FeedGenerator()
root = Path(deathbeds.__file__).parent
paths = list(root.glob('[0-9]*.ipynb'))
feed.id(str(root))
feed.title(deathbeds.__name__)
feed.author({'name': 'deathbeds'})
feed.link(href=str(root))

for path in paths:
    stats = path.stat()
    entry = feed.add_entry()
    entry.id(str(path))
    entry.title(' '.join(path.name.split('-')[3:]))
    entry.author( {'name':'deathbeds'} )
    entry.link(href=str(path))
    entry.description(summarize(path, ratio=.05).summary)
    entry.updated(pandas.to_datetime(stats.st_mtime_ns, utc=True).to_pydatetime())

posts = feedparser.parse(feed.atom_str())

C:\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2963: UserWarning: Discarding nonzero nanoseconds in conversion
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Anaconda3\lib\site-packages\nbconvert\filters\datatypefilter.py:41: UserWarning: Your element with mimetype(s) dict_keys(['application/literallycanvas+json']) is not able to be represented.
  mimetypes=output.keys())


In [4]:
df = pandas.Series(posts['entries']).apply(pandas.Series)

In [5]:
df.sample(2)

,id,guidislink,link,title,title_detail,updated,updated_parsed,authors,author_detail,author,content,summary,links
45,c:\users\deathbeds\deathbeds.github.io\deathbe...,True,c:\users\deathbeds\deathbeds.github.io\deathbe...,Tracing edges of image text.ipynb,"{'type': 'text/plain', 'language': None, 'base...",2018-07-19T17:10:30.512923+00:00,"(2018, 7, 19, 17, 10, 30, 3, 200, 0)",[{'name': 'deathbeds'}],{'name': 'deathbeds'},deathbeds,"[{'type': 'text/plain', 'language': None, 'bas...",* plot the outlines over top of the labelled i...,[{'href': 'c:\users\deathbeds\deathbeds.github...
28,c:\users\deathbeds\deathbeds.github.io\deathbe...,True,c:\users\deathbeds\deathbeds.github.io\deathbe...,Literate Markdown Kernel.ipynb,"{'type': 'text/plain', 'language': None, 'base...",2018-07-26T19:52:31.588995+00:00,"(2018, 7, 26, 19, 52, 31, 3, 207, 0)",[{'name': 'deathbeds'}],{'name': 'deathbeds'},deathbeds,"[{'type': 'text/plain', 'language': None, 'bas...",from deathbeds.__Literate_Markdown_Kernel impo...,[{'href': 'c:\users\deathbeds\deathbeds.github...


In [6]:
    blob = lambda x: """# [{title}]({link})
    
    ---
    {summary}
    
    ---""".format(**x.to_dict())

In [9]:
    def cap(object):
        out = W.Output()
        with out: display(Markdown(object))
        return out

In [12]:
with Sidecar():
    acc = W.Accordion(children=df.apply(blob, axis =1 ).apply(cap).tolist())
    [acc.set_title(*object) for object in enumerate(df.title)] and acc
    display(acc)